# Part I: Introduction

## 1. 用Generative和Discriminative Model处理文本分类问题
**文本分类问题：** input X是sentence，label y是该sentence的类型标签。求$\hat{y}=\underset{y}{argmax}P(y|X)$ \
**关键：** 找到$P(y|X)$的分布形态 \
**方法：** \
· 概率方法：生成模型和判别模型（本节主要关注这两种方法的差异） \
· 非概率方法：SVM，Perceptron，NN不用softmax

### 1.1 生成模型和判别模型
**<font color=orange>Generative model</font>**
1. 模型训练时：用贝叶斯分析方式对$𝑃(𝑦|𝑋)$做分解，转化为对先验分布$P(y_i;φ)$和似然$P(X|y_i;θ)$建模，用MLE或者CE做目标函数，求解参数θ。\
贝叶斯分析：\
$P(y_i|X) = \frac{P(X|y_i)*P(y_i)}{P(X)} = \frac{P(X|y_i)*P(y_i)}{Σ_{y_j\in{Y}} P(X|y_j)*P(y_j)}$ \
求解MLE：\
$\hat{θ}=\underset{θ,φ}{argmax}\ L(θ,φ) = \underset{θ,φ}{argmax}\ \frac{P(X|y_i;θ)*P(y_i;φ)}{Σ_{y_j\in{Y}} P(X|y_j;θ)*P(y_j;φ)}$
2. 模型推理时：同样利用贝叶斯关系对目标函数做分析，求解优化函数。 \
利用贝叶斯分析： \
$\hat{y}=\underset{y}{argmax}P(y|X) = \underset{y}{argmax}\frac{P(X|y)*P(y)}{P(X)}$ \
<font color=red>由于y的取值与P(X)无关</font>，因此有：\
$\hat{y}=\underset{y}{argmax}\ P(X|y;\hat{θ})*P(y;\hat{φ})=\underset{y}{argmax}\ P(X,y;\hat{θ},\hat{φ})$

**<font color=orange>Discriminative model</font>**
1. 模型训练时：直接对$P(y|X)$的参数建模，用MLE或者CE做目标函数，求解参数θ:
$\hat{θ}=\underset{θ}{argmax}\ L(θ) = \underset{θ}{argmax}\ \underset{i}{Σ}logP(y_{i}|X_{i};θ)$
2. 模型推理时：直接用估计的条件概率分布求概率最大的类型： \
$\hat{y}=\underset{y}{argmax}P(y|X;\hat{θ})$

### 1.2 用生成模型处理文本分类问题

#### 1.2.1 用生成模型解决问题的基础：语言模型
1. 生成模型在将后验概率转化为先验\*似然之后，实际应用场景下，比如分类问题中，一般会直接用训练集统计先验概率$P(y_i)$。然后分别对各个不同分类的样本集单独训练$P(X|y_i)$。从单个分类数据集而言，可以简写为$P(X)$
2. 在自然语言处理的语境中，X就是sentence。计算P(X)也就是计算sentence发生的概率。

<font color=blue>**语言模型：** models that assign probabilities to sequences of words.</font> \
· 可以用条件概率公式表达为：$P(X)= \prod_{i=1}^{I}P(x_i|x_1, x_2, ···,x_{i-1}) $  ，I是sentence长度  \
· <font color=red>求解语言模型的关键是估计$P(x_i|x_1, x_2, ···,x_{i-1})$</font>，<font color=green>**最简单的估计方法是n-gram models.**</font>

#### 1.2.2 最简单的语言模型：count-based unigram model
**特点：**
1. 独立性假设：$P(x_i|x_1, x_2, ···, x_{i-1})=P(x_i)$
2. 使用count-based MLE：<font color=red>count-based MLE是用MLE做估计得到的结论。</font> \
$P_{MLE}(x_i)=\frac{C_{train}(x_i)}{Σ_{\tilde{x}}C_{train}(\tilde{x})}$


**问题：** unknown words \
unknown words在training data中没有，因此count=0，$P(x_{unk})=0$。此时会导致Likelihood=0

**处理方式：** 需要一种分布，对所有可能的words所赋予的概率都大于0.
1. character/subword-based model：这是最常用的处理方法
2. smoothing，如，Laplace smoothing：实际上无效 \
· 思想：在训练模型时，给所有vocabulary的count都加1，因此unknown words的count=1。因此：\
$adjusted\ P(x_i)=\frac{C_{train}(x_i)+1}{Σ_{\tilde{x}}(C_{train}(\tilde{x})+1)} = \frac{C_{train}(x_i)+1}{Σ_{\tilde{x}}C_{train}(\tilde{x})+|V|}$ \
· 问题：unknown words的数量会很大，vocabulary要考虑所有unknown words的话，会导致$|V| \to \infty$,从而使$adjusted\ P(x_i)\to 0$
3. uniform distribution：基本不用 \
· 思想：在训练模型时，假设|V|固定，并让所有的unknown words的概率为$P_{unk}=\frac{1}{N_{|V|}}$

#### 1.2.3 BOW(Bag-of-words)生成模型: 又称为Naive Bayes
![BOW_generative_model](./pics/BOW_generative_model.png)
图中$θ_y$是在trainning set中计算的count-based probability。 \
用公式表达为：$P(X,y)=P(y)* {\textstyle \prod_{i=1}^{|X|}}P(x_i|y)=\frac{c(y)}{Σ_{y_j}c(y_j)}  {\textstyle \prod_{i=1}^{|X|}}\frac{c(x_i, y)}{Σ_{x_k}c(x_k, y)}$

#### 1.2.4 generative classifier与discriminate classifier的简单比较
generative models的求解方式很绕，为了求P(y|X)要先求P(X|y),而disciminative model则直接求P(X|y)。但Distriminative model不能使用简单的count-based decomposition。

### 1.3 用判别模型解决文本分类问题的方法
判别模型的训练思路：$\hat{θ}=\underset{θ}{argmax}\ L(θ) = \underset{θ}{argmax}\  {\textstyle \sum_{i=1}^{|X|}}\ logP(y_{i}|X_{i};θ)
$

#### 1.3.1 BOW(Bag-of-words)判别模型
**二分类问题**：先计算score，再用logistic function转化为probability
1. 模型训练阶段：\
用MLE给每个word $x_i$找到$θ_{y|x_i}=P(y|x_i)$ \
这里minL(θ)用Gradient Descent
2. 模型推理阶段： \
先计算score：$score_{y|X}=θ_y+{\textstyle \prod_{i=1}^{|X|}}θ_{y|x_i} 
$ \
再转化为probability：$P(y|X;θ)=sigmoid(score_{y|X})=\frac{1}{1+e^{-score_{y|X}}}$

**多分类问题**：先计算score，再用softmax function转化为probability


## 2. 模型评估
### 2.1 指标
1. Accuracy：$acc(y,\hat y)=\frac{1}{|y|}\sum_{i=1}^{|y|} Count(y_i=\hat{y}_i)$ \
<font color=red>问题：如果有出现概率非常小的类型，比如P(y=0)=1%，那么分类器只要全判y=1就有acc=99% </font>
2. precision（<font color=orange>**识别出来的目标的正确率**</font>）：估$\hat{y}=1$的样本中，确实是y=1的比例 \
$prec(y, \hat{y}) = \frac{c(y=1, \hat{y}=1)}{c(\hat{y}=1)} $
3. recall（<font color=orange>**有多少比例的目标被识别**</font>）：所有y=1的样本中，被估为$\hat{y}=1$的比例\
$rec(y, \hat{y}) = \frac{c(y=1, \hat{y}=1)}{c(y=1)} $
4. F-measure(F1 score)：$\frac{1*prec*rec}{prec+rec}$

### 2.2 统计检验(significant testing)的基本概念
1. p-value:
2. confidence interval

### 2.3 unpaired and paired test

### 2.4 bootstrap test